Data QA Case: ``mggg-states``
========================

Below are the steps involved in performing automated data quality checks on ``mggg-states`` data. Specifically, this notebook compares the vote counts in ``mggg-states`` with those in MEDSL's and Wikipedia's datasets for the following elections:

- 2016 United States presidential election
- 2016 United States Senate elections
- 2016 United States House of Representatives elections
- 2017 United States Senate elections (special elections)
- 2018 United States Senate elections
- 2018 United States House of Representatives elections

*Note:* the automated checks are not completely exhaustive and further manual checks are required.

Step 0. Setup
----------------

In [1]:
# !pip3 install numpy
# !pip3 install pandas
# !pip3 install geopandas
# !pip3 install wikipedia

# !pip3 install git+https://github.com/KeiferC/gdutils.git

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import wikipedia
import os
import re

import gdutils.datamine as dm
import gdutils.dataqa as dq
import gdutils.extract as et

from typing import Any, List, Tuple, Dict, Hashable, Union, NoReturn

Step 1. Data collection
---------------------------

In [3]:
state_names = [
    'Alabama', 'Alaska','Arizona', 'Arkansas', 'California', 
    'Colorado', 'Connecticut', 'Delaware',  'Florida', 'Georgia', 
    'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
    'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 
    'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 
    'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 
    'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 
    'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 
    'VA', 'WA', 'WV', 'WI', 'WY']

states = list(zip(state_names, state_abbreviations))

__Step 1.1.__ Gather ``mggg-states`` data

In [4]:
# commented out to save local space and time
# # this will take some time to complete
# dm.clone_gh_repos(account='mggg-states', account_type='orgs', 
#                   outpath=os.path.join('src', 'mggg'))

In [5]:
mggg_gdfs = {}

# stores extracted GeoDataFrames in a dictionary where the keys are the filenames from
# which the GeoDataFrames were found
for filepath in dm.list_files_of_type('.zip', os.path.join('src', 'mggg')):
      mggg_gdfs[os.path.basename(filepath)[:-4]] = et.read_file(filepath).extract()

__Step 1.2.__ Gather MEDSL data for comparison purposes

In [6]:
# Print available MEDSL data to select applicable datasets

# # commented out to save local space and time
# print('{:27} : {}'.format('Repo Name', 'Repo URL'))
# print('------------------------------------------------------------------')
# for (repo, url) in dm.list_gh_repos(account='MEDSL', account_type='orgs'):
#     print("{:27} : {}".format(repo, url))

In [7]:
medsl_repos = ['official-precinct-returns', # precinct-level 2016 election results
               '2018-elections-official']   # constituency-level 2018 election results

# commented out to save local space and time
# # this will take some time to complete
# dm.clone_gh_repos(account='MEDSL', account_type='orgs', repos=medsl_repos, 
#                   outpath=os.path.join('src', 'medsl'))

In [8]:
medsl_dfs = {}

# stores extracted GeoDataFrames in a dictionary where the keys are the filenames from
# which the GeoDataFrames were found
for filepath in dm.list_files_of_type('.zip', os.path.join('src', 'medsl')):
    medsl_dfs[os.path.basename(filepath)[:-4]] = et.read_file(filepath).extract()

__Step 1.3.__ Gather Wikipedia data for comparison purposes (see ``src/wikipedia-election-data-mining.ipynb`` for more details on mining Wikipedia data)

In [9]:
# TODO: coming soon

[Coming Soon] __Step 1.4.__ Gather Ballotpedia data for comparison purposes

*Note:* Depending on response to API access, this step may need to be done manually

Step 2. Data wrangling
---------------------------

__Step 2.1.__ Wrangle MEDSL data

In [10]:
def pivot_medsl(medsl_dfs_dict: Dict[str, Union[pd.DataFrame, gpd.GeoDataFrame]]
        ) -> Dict[str, Union[pd.DataFrame, gpd.GeoDataFrame]]:
    """
    Given a dictionary of MEDSL (Geo)DataFrames, returns a dictionary of
    pivoted DataFrames where the columns are the election and parties and 
    the values are the votes for every precinct.
    
    """
    for df in medsl_dfs_dict:
        medsl_pvt = medsl_dfs_dict[df].pivot_table(index='precinct',
                                                   columns=['office', 'party'],
                                                   values='votes')
        medsl_pvt.columns = [' '.join(col).strip() for col in medsl_pvt.columns.values]
        medsl_dfs_dict[df] = et.ExtractTable(medsl_pvt).extract()

In [11]:
# View available data to find applicable datasets

# commented out to save screen space
# for df in medsl_dfs:
#     print('--------{}--------'.format(df))
#     print(medsl_dfs[df].head())
#     print()

In [12]:
# Pivot and extract relevant MEDSL election data

medsl_18_dfs = {}
medsl_pres16_dfs = {}
medsl_sen16_dfs = {}
medsl_ush16_dfs = {}

for state in states:
    st, st_abv = state
    try:
        medsl_18_dfs[st] = et.ExtractTable(medsl_dfs['precinct_2018'], 
                                           column='state', value=st).extract()
    except Exception as e:
        print('Missing data in medsl_18:', e)
    
    try:
        medsl_pres16_dfs[st] = et.ExtractTable(medsl_dfs['2016-precinct-president'], 
                                               column='state', value=st).extract()
    except Exception as e:
        print('Missing data in medsl_pres16:', e)
    
    try:
        medsl_sen16_dfs[st] = et.ExtractTable(medsl_dfs['2016-precinct-senate'], 
                                              column='state', value=st).extract()
    except Exception as e:
        print('Missing data in medsl_sen16:', e)
    
    try:
        medsl_ush16_dfs[st] = et.ExtractTable(medsl_dfs['2016-precinct-house'], 
                                              column='state', value=st).extract()
    except Exception as e:
        print('Missing data in medsl_ush16:', e)

        
pivot_medsl(medsl_18_dfs)
pivot_medsl(medsl_pres16_dfs)
pivot_medsl(medsl_sen16_dfs)
pivot_medsl(medsl_ush16_dfs)

Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Delaware'"
Missing data in medsl_18: Initialization failed. "Column 'state' has no value 'Indiana'"
Missing data in medsl_18: Initialization failed. "Column 'state' has no value 'Kentucky'"
Missing data in medsl_18: Initialization failed. "Column 'state' has no value 'Maine'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Maine'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Massachusetts'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Michigan'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Minnesota'"
Missing data in medsl_18: Initialization failed. "Column 'state' has no value 'Mississippi'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no value 'Mississippi'"
Missing data in medsl_sen16: Initialization failed. "Column 'state' has no va

__Step 2.2.__ Wrangle Wikipedia data

Step 3. Data standardization check of ``mggg-states``
---------------------------------------------------------------

__Step 3.1__ Generate naming conventions

In [13]:
with open(os.path.join('src', 'naming_convention.json')) as json_file:
    standards_raw = json.load(json_file)
    
offices = dm.get_keys_by_category(standards_raw, 'offices')
parties = dm.get_keys_by_category(standards_raw, 'parties')
counts  = dm.get_keys_by_category(standards_raw, 'counts')
others  = dm.get_keys_by_category(standards_raw, ['geographies', 
                                                  'demographics', 
                                                  'districts', 
                                                  'other'])

elections = [office + format(year, '02') + party 
             for office in offices
             for year in range(0, 21)
             for party in parties 
             if not (office == 'PRES' and year % 4 != 0)]

counts    = [count + format(year, '02') 
             for count in counts 
             for year in range(0, 20)]

standards = elections + counts + others

__Step 3.2.__ Check ``mggg-states`` data compliance with naming conventions

In [14]:
naming_check = {}

for gdf in mggg_gdfs:
      naming_check[gdf] = dq.compare_column_names(mggg_gdfs[gdf], standards)

In [15]:
# Print and store results of naming convention check

# dictionary with mggg GeoDataFrame names as keys (names of files from
# which dataset was gathered), and a set of columns that fit the 
# naming convention as values
matched_columns = {}

for gdf in naming_check:
    print('=========================================')
    print('Dataset: {}'.format(gdf))
    print('=========================================')
    
    (matches, diffs) = naming_check[gdf]
    matched_columns[gdf] = matches
    
    diffs = list(diffs)
    diffs.sort()
    
    print('Discrepancies from naming convention:')
    print(diffs)
    print()

Dataset: MN12
Discrepancies from naming convention:
['CA1NO12', 'CA1YES12', 'CA2NO12', 'CA2YES12', 'CONGDIST', 'COUNTYFIPS', 'COUNTYNAME', 'CTU_TYPE', 'CTYCOMDIST', 'JUDDIST', 'MCDCODE', 'MCDNAME', 'MNLEGDIST', 'MNSENDIST', 'PCTCODE', 'PCTNAME', 'VTD']

Dataset: MN16
Discrepancies from naming convention:
['CONGDIST', 'COUNTYFIPS', 'COUNTYNAME', 'CTU_TYPE', 'CTYCOMDIST', 'JUDDIST', 'MCDCODE', 'MCDNAME', 'MNLEGDIST', 'MNSENDIST', 'PCTCODE', 'PCTNAME', 'VTDID']

Dataset: MN14
Discrepancies from naming convention:
['AG14LM', 'AUD14LC', 'CONGDIST', 'COUNTYFIPS', 'COUNTYNAME', 'CTU_TYPE', 'CTYCOMDIST', 'JUDDIST', 'MCDCODE', 'MCDNAME', 'MNLEGDIST', 'MNSENDIST', 'PCTCODE', 'PCTNAME', 'VTDID']

Dataset: MN12_18
Discrepancies from naming convention:
['AG14LM', 'AG18LC', 'AUD14LC', 'AUD18LM', 'CA1NO12', 'CA1YES12', 'CA2NO12', 'CA2YES12', 'CONGDIST', 'COUNTYFIPS', 'COUNTYNAME', 'CTU_TYPE', 'CTYCOMDIST', 'GOV18LC', 'JUDDIST', 'MCDCODE', 'MCDNAME', 'MNLEGDIST', 'MNSENDIST', 'PCTCODE', 'PCTNAME', 'SE

Step 4. Compare ``mggg-states`` data with external sources
----------------------------------------------------------

In [16]:
# Categorize mggg-states dataframes by the names of their States

# a list of tuples of state names (left) and a list of GeoDataFrames (right)
available_mggg_states = []

for state in states:
    state_name, state_abv = state
    
    # messy name matching because file naming isn't standardized
    mggg_gdf_names = [gdf_name for gdf_name in list(mggg_gdfs) 
                               if gdf_name.startswith(state_abv.lower() + '_') or
                                  gdf_name.startswith(state_abv + '_') or
                                  gdf_name.startswith(state_name.lower() + '_') or
                                  gdf_name.startswith(state_name.upper() + '_') or
                                  gdf_name.startswith(state_name + '_') or
                                  gdf_name.startswith(
                                      state_name.replace(' ', '_').lower() + '_') or
                                  gdf_name.startswith(
                                      state_name.replace(' ', '_').upper() + '_') or
                                  gdf_name.startswith(
                                      state_name.replace(' ', '_') + '_')]
                      
    available_mggg_states.append((state_name, mggg_gdf_names))

In [17]:
# View available columns that can be compared in mggg-states

# commented out to save screen space
# for state in available_mggg_states:
#     st, gdf_names = state
#     if gdf_names:
#         print('======== {} ========'.format(st))
#         for name in gdf_names:
#             print('{} --------'.format(name))
#             print(sorted(list(matched_columns[name])))
#             print()
#         print()

__Step 4.1.__ Compare against MEDSL

In [18]:
# Generate Naming Convention Translations between MGGG and MEDSL
# Has to be done manually because some MEDSL data don't use the 
# same naming convention

pres16_cols = [
    ('PRES16D', 'US President democratic'), 
    ('PRES16G', 'US President green'), 
    ('PRES16L', 'US President libertarian'), 
    ('PRES16R', 'US President republican')
]

sen16_cols = [
    ('SEN16D', 'US Senate democrat'),
    ('SEN16G', 'US Senate green'),
    ('SEN16L', 'US Senate libertarian'),
    ('SEN16R', 'US Senate republican')
]

ush16_cols = [
    ('USH16D', 'US House democratic'),
    ('USH16G', 'US House green'),
    ('USH16L', 'US House libertarian'),
    ('USH16R', 'US House republican')
]

fed18_cols = [
    ('SEN18D', 'US Senate democratic'),
    ('SEN18G', 'US Senate green'),
    ('SEN18L', 'US Senate libertarian'),
    ('SEN18R', 'US Senate republican'),
    ('USH18D', 'US House democrat'),
    ('USH18G', 'US House green'),
    ('USH18L', 'US House libertarian'),
    ('USH18R', 'US House republican')
]

In [19]:
def bulk_compare(bulk_results: Dict[str, List[Tuple[Hashable, Any]]], 
                 st: str, 
                 mggg_names: List[str], 
                 medsls: List[pd.DataFrame], 
                 cols: Tuple[str, str]
        ) -> NoReturn: # Returns bulk_results by reference
    """
    Returns, by reference, a dict containing state names as keys and a value of a 
    dict containing mggg_gdf names as keys and the results of dm.compare_column_sums
    as values.
    
    """
    if st in medsls:
        try:
            mggg_results = bulk_results[st]
        except Exception:
            mggg_results = {}
            
        for mggg_name in mggg_names:
            x = []
            y = []
            
            for mggg_col, medsl_col in cols:
                if (mggg_col in list(mggg_gdfs[mggg_name]) and 
                    medsl_col in list(medsls[st])):
                    x.append(mggg_col)
                    y.append(medsl_col)
                
            if x:
                try:
                    # append results
                    mggg_results[mggg_name] += \
                        dq.compare_column_sums(mggg_gdfs[mggg_name], 
                                               medsls[st], x, y)
                except Exception as e:
                    try: # if results don't already exist
                        mggg_results[mggg_name] = \
                            dq.compare_column_sums(mggg_gdfs[mggg_name], 
                                                   medsls[st], x, y)
                    except Exception as z:
                        print("Unable to compare {} and {} in {}.\n\t{}".format(
                              x, y, mggg_name, z))
            else:
                mggg_results[mggg_name] = []
        
            bulk_results[st] = mggg_results

In [20]:
# Compare mggg-states data with MEDSL data

# a dictionary with State names as keys and with a list 
# of comparison results of column for each mggg-states
# dataframe (dicts) as values
results = {}

# printed errors indicate differing column data types --
# to manually investigate as part of QA
for state in available_mggg_states:
    st, mggg_names = state
    
    if mggg_names:
        bulk_compare(results, st, mggg_names, medsl_pres16_dfs, pres16_cols)
        bulk_compare(results, st, mggg_names, medsl_sen16_dfs, sen16_cols)
        bulk_compare(results, st, mggg_names, medsl_ush16_dfs, ush16_cols)
        bulk_compare(results, st, mggg_names, medsl_18_dfs, fed18_cols)

Unable to compare ['PRES16D', 'PRES16L', 'PRES16R'] and ['US President democratic', 'US President libertarian', 'US President republican'] in GA_precincts.
	ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U8310'), dtype('<U8310')) -> dtype('<U8310')
Unable to compare ['SEN16L', 'SEN16R'] and ['US Senate libertarian', 'US Senate republican'] in GA_precincts.
	ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U5508'), dtype('<U5508')) -> dtype('<U5508')
Unable to compare ['PRES16D', 'PRES16R'] and ['US President democratic', 'US President republican'] in MA_no_islands_12_16.
	ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U7939'), dtype('<U7939')) -> dtype('<U7939')
Unable to compare ['SEN16R'] and ['US Senate republican'] in VT_towns.
	ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U747'), dtype('<U747')) -> dtype('<U747')


In [21]:
# Print comparison results

print("============================================================================")
print("Results of State-level Aggregation comparisons between mggg-states and MEDSL")
print("============================================================================")
print()
print('{:37} : {}'.format('mggg-states column [vs] MEDSL column', 'difference in sums'))
print('----------------------------------------------------------------------------')

for st in results:
    print("######## {} ########".format(st))
    
    for mggg_name in results[st]:
        print("{} ========".format(mggg_name))
        
        if results[st][mggg_name]:
            for col_v_col, diff in results[st][mggg_name]:
                print('{:37} : {}'.format(col_v_col, diff))
        else:
            print("No comparable columns found.")
            
        print()
    print()


Results of State-level Aggregation comparisons between mggg-states and MEDSL

mggg-states column [vs] MEDSL column  : difference in sums
----------------------------------------------------------------------------
######## Alaska ########
AK_precincts ========
PRES16D [vs] US President democratic  : -47220.0
PRES16G [vs] US President green       : -1947.5
PRES16L [vs] US President libertarian : -6717.5
PRES16R [vs] US President republican  : -59910.0
SEN16L [vs] US Senate libertarian     : -30050.0
SEN16R [vs] US Senate republican      : -50259.5
USH16D [vs] US House democratic       : -43628.5
USH16L [vs] US House libertarian      : -11665.5
USH16R [vs] US House republican       : -54359.5
USH18R [vs] US House republican       : -46390.5


######## Arizona ########
az_precincts ========
SEN18R [vs] US Senate republican      : 0.0
USH18D [vs] US House democrat         : 0.0
USH18G [vs] US House green            : -3672.0
USH18R [vs] US House republican       : 0.0


######## Colorado #

__Don't panic just yet -- potential explanations for differences:__

- MGGG data may have columns with mixed datatypes. Some columns contain strings, when they should contain numbers
- Some shapefiles use VTDs and towns, which may differ from precincts
- Undercounts may be the result of excluding absentee ballots in the count
- Sums should be spot-checked manually in addition to the automated checks
- The comparisons are conducted on the intersection between mggg-states and MEDSL datasets and are limited to 2016 and 2018 data
- MEDSL float values indicate proration -- to investigate
- I also haven't rigourously examined the MEDSL data :P

[Coming Soon] __Step 4.2.__ Compare against Wikipedia

[Coming Soon] __Step 4.3.__ Compare against Ballotpedia

__Step 4.4__ Data summation for manual checks against external sources

In [22]:
# Print MGGG summation results

# commented out to save screen space -- TODO: uncomment final
# print("============================================================================")
# print("Results of State-level Aggregations - mggg-states")
# print("============================================================================")
# print()
# print('{:11} : {:10}\t\t{}'.format('mggg-states column', 'sum', 'dtype'))
# print('----------------------------------------------------------------------------')

# for st, gdf_names in available_mggg_states:
#     if gdf_names:
#         print('######## {} ########'.format(st))
        
#         for gdf_name in gdf_names:
#             print("{} ========".format(gdf_name))

#             cols_to_sum = [col for col in matched_columns[gdf_name] if col != 'geometry']
#             col_sums = dq.sum_column_values(mggg_gdfs[gdf_name], cols_to_sum)

#             for result in col_sums:
#                 col_name, col_sum = result
                
#                 if col_name in matched_columns[gdf_name]:
#                     if isinstance(col_sum, str):
#                         col_sum = "{} is a str".format(col_name)
                        
#                     print('{:11} : {:10}\t\t{}'.format(col_name, col_sum, str(type(col_sum))))
                    
#             print('\n\n')
    

In [23]:
# Print MEDSL summation results

def print_medsl_sums(st: str, 
                     medsl_name: str, 
                     medsls: Dict[str, Union[pd.DataFrame, gpd.GeoDataFrame]]
        ) -> NoReturn:
    if st in medsls:
        print("######## {} : {} ########".format(st, medsl_name))
        
        cols_to_sum = [col for col in list(medsls[st]) if col != 'geometry']
        
        try:
            col_sums = dq.sum_column_values(medsls[st], cols_to_sum)
        except Exception as e:
            pass
        
        for result in col_sums:
            col_name, col_sum = result
            print('{:65} : {}\t{}'.format(col_name, col_sum, str(type(col_sum))))

        print('\n\n')

# commented out to save screen space -- TODO: uncomment final        
# print("============================================================================")
# print("Results of State-level Aggregations - MEDSL")
# print("============================================================================")
# print()
# print('{:65} : {}\t{}'.format('MEDSL column', 'sum', 'dtype'))
# print('----------------------------------------------------------------------------')

# for st, st_abv in states:
#     print_medsl_sums(st, 'MEDSL 2018', medsl_18_dfs)
#     print_medsl_sums(st, 'MEDSL PRES16', medsl_pres16_dfs)
#     print_medsl_sums(st, 'MEDSL SEN16', medsl_sen16_dfs)
#     print_medsl_sums(st, 'MEDSL USH16', medsl_ush16_dfs)
    

Step 5. Check topological soundness of ``mggg-states`` data
-----------------------------------------------------------------------

__Step 5.1.__ Check for empty or missing geometries

In [24]:
topological_warnings = []
for gdf in mggg_gdfs:
    if any(mggg_gdfs[gdf]['geometry'].isna()):
        topological_warnings.append('{} has missing geometries.'.format(gdf))
        
    if any(mggg_gdfs[gdf]['geometry'].is_empty):
        topological_warnings.append('{} has empty geometries.')

if len(topological_warnings) == 0:
    print("No missing or empty geometries.")
else:
    [print(msg) for msg in topological_warnings]

No missing or empty geometries.


Step 6. Cleanup
-------------------

In [25]:
# Remove cloned repos

# commented out to save local space and time
# dm.remove_repos('src/')

In [26]:
# Uninstall installed python packages

# commented out to save local space and time
# !echo y | pip3 uninstall numpy
# !echo y | pip3 uninstall pandas
# !echo y | pip3 uninstall geopandas
# !echo y | pip3 uninstall wikipedia

# !echo y | pip3 uninstall gdutils

In [27]:
# Reset Jupyter Notebook IPython Kernel

# commented out to save local space and time
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Next Steps
-------------

__Data Standardization__

- Manually evaluate column naming discrepancies to determine if changes are needed
- Manually evaluate column datatypes to determine if changes are needed

__Data Comparison__

- Manually investigate large differences found through comparing ``mggg-states`` data with external sources (e.g. Are absentee ballots counted? Are the precinct counts accurate? etc.) 
- For more accurate comparisons, compare ``mggg-states`` data with those in each States' Secretary of State website

__Topological Soundness__

- Manually examine shapefiles for gaps and overlaps. *Note:* although gaps and overlaps are not necessarily indicators of inaccurate data (because some counties have precinct islands), they do mean that the data cannot be for chain runs. 

__Data Documentation__

- Do the READMEs provide data sources?
- Do the READMEs describe what aggregation/disaggregation processes were used?
- Do the READMEs discuss discrepancies/caveats in the data?
- Do the READMEs provide scripts used and/or discuss the data wrangling/processing process?